## Import
Use elements of the scientific python stack

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr

# Import ast which helps with parsing the list of variables
import ast

# Import intake-esm
import intake

# Import dask
import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

%matplotlib inline

## Setup a PBSCluster
Speed up processing with DASK with 8 threads

In [ ]:
cluster = PBSCluster(
    cores=1, # The number of cores you want
    memory='10GB', # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus=1:mem=10GB', # Specify resources
    project='CESM0002', # Input your project ID here
    walltime='04:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)

cluster.scale(8) # Scale up

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [ ]:
client

## Open a catalog

In [ ]:
cat = intake.open_esm_datastore(
    "/glade/work/marsh/intake-esm-catalogs/WACCM6-MA-1deg.json",
    ).search(
    variable=('Uzm'),
    frequency=('day_1')
)
print(cat.keys())
cat

In [ ]:
dsets = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time': 8}, 'decode_times': False})

In [ ]:
dsets.keys()

In [ ]:
k = 'atm.cam.h6.b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.002'
print(k)

ds = dsets[k]
ds = xr.decode_cf(ds)
print(ds['time'][0].values)
ds = ds.sortby('time')
print(ds['time'][0].values)

u0 = ds['Uzm'].sel(lat=0.0, method='nearest').sel(time=slice("1980-01", "2010-12"))


In [ ]:
fig = u0[:,:,0].plot.contourf(y='ilev', 
                              ylim=(100,1), 
                              yincrease=False, 
                              yscale='log',
                              figsize=(15,6), 
                              levels=np.arange(-30,31,2.5),
                              )

fig.axes.set_ylabel('Pressure (hPa)')
fig.axes.set_xlabel('Year')
plt.title('Equatorial zonal mean U')

plt.show()

In [ ]:
# data looks reasonable as values
plt.figure(figsize=(15,6))

t1.plot()

plt.show()

In [ ]:
u1

In [ ]:
# time recognized when a single time is extracted
u0 =  ds['Uzm'].isel(time=180, zlon=0)
u0

In [ ]:
fig = u0.plot(cmap='RdYlBu_r', figsize=(9,6), 
              ylim=(1000,0.1), 
              vmin=-60, vmax=60
              )
u0.plot.contour(yincrease=False, y='ilev', colors='black', levels=np.arange(-90,91,15))

fig.axes.set_yscale('log')
fig.axes.set_ylabel('Pressure (hPa)')
plt.show()

In [ ]:
# time recognized when a single time is extracted
t0 =  ds['T'].isel(time=31).mean(dim='lon')
fig = t0.plot(cmap='RdYlBu_r', figsize=(9,6), 
              ylim=(1000,0.0001), 
              vmin=110, vmax=330
              )
t0.plot.contour(yincrease=False, colors='black', levels=np.arange(110,350,15))

fig.axes.set_yscale('log')
fig.axes.set_ylabel('Pressure (hPa)')
plt.show()

In [ ]:
col.nunique()

In [ ]:
hist_cases = [key for key in list(cat) if "HIST" in key]
print(hist_cases)

In [ ]:
hist_cases?

In [ ]:
cluster.scale(0)